# Deep Learning Explore. 

In [1]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Crear generadores de datos. 

In [3]:
import os
import shutil
import random

# Directorio donde están tus imágenes originales
original_data_dir = "../data/raw/train" 

# Directorio de destino para la nueva estructura
base_dir = "../data/raw/dataset"

# Porcentaje de imágenes para el conjunto de validación
validation_split = 0.2

# Crear la estructura de carpetas
try:
    os.makedirs(os.path.join(base_dir, 'train', 'cat'))
    os.makedirs(os.path.join(base_dir, 'train', 'dog'))
    os.makedirs(os.path.join(base_dir, 'validation', 'cat'))
    os.makedirs(os.path.join(base_dir, 'validation', 'dog'))
    print("Estructura de carpetas creada exitosamente.")
except FileExistsError:
    print("La estructura de carpetas ya existe. Continuando...")

# Obtener la lista de todos los archivos de imágenes
file_names = os.listdir(original_data_dir)
random.shuffle(file_names) # Mezclar la lista para una división aleatoria

# Dividir los archivos en conjuntos de entrenamiento y validación
split_index = int(len(file_names) * (1 - validation_split))
train_files = file_names[:split_index]
validation_files = file_names[split_index:]

print(f"\nTotal de imágenes encontradas: {len(file_names)}")
print(f"Imágenes para entrenamiento: {len(train_files)}")
print(f"Imágenes para validación: {len(validation_files)}")

# Mover archivos al directorio de entrenamiento
print("\nMoviendo archivos de entrenamiento...")
for file_name in train_files:
    if file_name.startswith('cat'):
        shutil.copy(os.path.join(original_data_dir, file_name), os.path.join(base_dir, 'train', 'cat', file_name))
    elif file_name.startswith('dog'):
        shutil.copy(os.path.join(original_data_dir, file_name), os.path.join(base_dir, 'train', 'dog', file_name))

# Mover archivos al directorio de validación
print("Moviendo archivos de validación...")
for file_name in validation_files:
    if file_name.startswith('cat'):
        shutil.copy(os.path.join(original_data_dir, file_name), os.path.join(base_dir, 'validation', 'cat', file_name))
    elif file_name.startswith('dog'):
        shutil.copy(os.path.join(original_data_dir, file_name), os.path.join(base_dir, 'validation', 'dog', file_name))

print("\n¡Organización de archivos completa!")

Estructura de carpetas creada exitosamente.

Total de imágenes encontradas: 25000
Imágenes para entrenamiento: 20000
Imágenes para validación: 5000

Moviendo archivos de entrenamiento...
Moviendo archivos de validación...

¡Organización de archivos completa!


In [4]:
# Definir las rutas de las carpetas
train_dir = '../data/raw/dataset/train'
validation_dir = '../data/raw/dataset/validation'

# Definir los parámetros para el preprocesamiento de imágenes
IMG_WIDTH = 200
IMG_HEIGHT = 200
BATCH_SIZE = 32

# Crear el objeto ImageDataGenerator para los datos de entrenamiento
# trdata (entrenamiento) con aumento de datos
trdata = ImageDataGenerator(
    rescale=1./255,           # Normalizar los valores de píxel
    rotation_range=40,        # Aumento de datos para el entrenamiento
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Crear el objeto ImageDataGenerator para los datos de validación/prueba
# tsdata (validación), solo escalado
tsdata = ImageDataGenerator(rescale=1./255)

# Pasa la carpeta de entrenamiento al objeto trdata
# El generador de entrenamiento etiquetará automáticamente las imágenes
train_generator = trdata.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Redimensionar a 200x200
    batch_size=BATCH_SIZE,
    class_mode='binary'                   # 'binary' para dos clases (gato, perro)
)

# Pasa la carpeta de validación al objeto tsdata
# El generador de validación etiquetará las imágenes de prueba
validation_generator = tsdata.flow_from_directory(
    validation_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Opcional: Imprimir las clases detectadas para verificar que todo esté bien
print("\nClases detectadas por el generador de entrenamiento:")
print(train_generator.class_indices)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.

Clases detectadas por el generador de entrenamiento:
{'cat': 0, 'dog': 1}
